## News Article Scraper

This code scrapes news articles from prominent news sources and saves them in a dictionary called "articles" for scoring later.

In [1]:
import string
import urllib.request
import httplib2
import requests
from requests.exceptions import ConnectionError, RequestException
from bs4 import BeautifulSoup, SoupStrainer
import csv
import time
import re  #https://regex101.com/


In [2]:
articles = dict()

In [3]:
# Define text-cleaning functions

def replace_punct(input):
    input = input.replace(',', '')
    input = input.replace('\'', '')
    input = input.replace(':', '')
    input = input.replace('[', '')
    input = input.replace(']', '')
    input = input.replace('(', '')
    input = input.replace(')', '')
    input = input.replace('_', '')
    input = input.replace('"', '')

    input = input.replace('-', ' ')
    input = input.replace('+', ' ')

    return input

def clean_article(text):

    # replace certain strings
    text = text.replace('expand / contract','')  

    text = text.replace('\n',' ')
    text = text.lower()
    text = re.sub(r'\\u[0-9]+[abcde]*', '',text) 

    text = replace_punct(text)
    text = re.sub(r'\d+', '{DG}', text) 

    return text

In [26]:
# For Fox News - this works!

def fox_scraper(url):
    r = urllib.request.urlopen(url)
    soup = BeautifulSoup(r, 'html.parser')
    tags = soup.find_all("div", class_="article-text")
    
    # Clean text
    article_text = re.sub("<[^>]*>", "", str(tags), flags=re.MULTILINE)
    article_text = article_text.replace('\xa0',' ')
    article_text = clean_article(article_text)
    
    return article_text

urls = [
    'http://www.foxnews.com/politics/2017/04/18/trump-signs-order-to-clamp-down-on-visa-program-enforce-buy-american-policy.html',
    'http://www.foxnews.com/us/2017/04/18/facebook-killer-steve-stephens-dead-police-say.html',
    'http://www.foxnews.com/opinion/2017/04/17/care-ceo-famine-is-stalking-globe-and-america-urgently-needs-to-lead.html',
    'http://www.foxnews.com/politics/2017/04/18/georgia-race-gop-hits-ossoff-for-living-outside-district-in-final-stretch.html',
    'http://www.foxnews.com/politics/2017/04/18/trump-illegal-immigrant-criminals-are-getting-hell-out.html',
    'http://www.foxnews.com/politics/2017/04/19/uss-carl-vinson-carrier-was-not-on-way-to-north-korea-reports-say.html',
    'http://www.foxnews.com/politics/2017/04/19/iran-nuclear-deal-trump-administration-says-tehran-complying-with-agreement.html',
    'http://www.foxnews.com/us/2017/04/19/death-ny-state-judge-found-in-hudson-river-reportedly-suspicious.html',
    'http://www.foxnews.com/us/2017/04/19/inspector-who-tried-to-rein-in-illegal-logging-was-fired.html',
    'http://www.foxbusiness.com/politics/2017/04/18/with-democratic-invasion-white-house-cuban-starting-to-warm-up-to-trump-presidency.html'
]

n = 0
for url in urls:
    articles['fox_%02d' % n] = fox_scraper(url)
    n += 1 
    


In [27]:
# For NYT - this works!

def nyt_scraper(url):
    response = requests.get(url, timeout=1.000)
    soup = BeautifulSoup(response.text, 'html.parser')   

    tag = soup.find_all('p', class_="story-body-text story-content", text=True)
    article_text = ''
    for t in tag:
        article_text += str(t)     
    
    # Cleaning text
    article_text = re.sub("<[^>]*>", "", str(article_text), flags=re.MULTILINE)
    article_text = clean_article(article_text)
    
    return article_text

urls = [
    'https://www.nytimes.com/2017/04/18/world/asia/aircraft-carrier-north-korea-carl-vinson.html?ref=todayspaper&_r=0',
    'https://www.nytimes.com/2017/04/18/world/europe/uk-theresa-may-general-election.html?ribbon-ad-idx=3&rref=todayspaper&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Today%E2%80%99s%20Paper&pgtype=article',
    'https://www.nytimes.com/2017/04/18/world/europe/hungary-orban-populism-migrants-border-european-union.html?ribbon-ad-idx=3&rref=todayspaper&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Today%E2%80%99s%20Paper&pgtype=article',
    'https://www.nytimes.com/2017/04/18/opinion/mr-trump-plays-by-his-own-rules-or-no-rules.html?action=click&pgtype=Homepage&clickSource=story-heading&module=opinion-c-col-left-region&region=opinion-c-col-left-region&WT.nav=opinion-c-col-left-region',
    'https://www.nytimes.com/2017/04/18/opinion/defending-governor-cuomos-free-tuition-plan.html?ribbon-ad-idx=16&rref=opinion&module=Ribbon&version=context&region=Header&action=click&contentCollection=Opinion&pgtype=article',
    'https://www.nytimes.com/2017/04/18/opinion/my-daughter-is-not-transgender-shes-a-tomboy.html?action=click&pgtype=Homepage&clickSource=story-heading&module=opinion-c-col-right-region&region=opinion-c-col-right-region&WT.nav=opinion-c-col-right-region',
    'https://www.nytimes.com/2017/04/19/us/politics/georgia-special-election.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news',
    'https://www.nytimes.com/2017/04/18/us/politics/georgia-special-election-jon-ossoff.html?ribbon-ad-idx=6&rref=homepage&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Home%20Page&pgtype=article',
    'https://www.nytimes.com/2017/04/18/business/media/fox-bill-oreilly.html?ribbon-ad-idx=6&rref=homepage&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Home%20Page&pgtype=article',
    'https://www.nytimes.com/2017/04/18/us/politics/executive-order-hire-buy-american-h1b-visa-trump.html?ribbon-ad-idx=6&rref=homepage&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Home%20Page&pgtype=article',
    'https://www.nytimes.com/2017/04/18/us/politics/trump-inauguration-fundraising.html?ribbon-ad-idx=6&rref=homepage&module=Ribbon&version=origin&region=Header&action=click&contentCollection=Home%20Page&pgtype=article&mtrref=www.nytimes.com&gwh=65A993E2EE2CFFC71965945E1349AC8C&gwt=pay'
]


n = 0
for url in urls:
    articles['nyt_%02d' % n] = nyt_scraper(url)
    n += 1 

In [28]:
# Washington Post - this works!

def post_scraper(url):
    r = urllib.request.urlopen(url)
    soup = BeautifulSoup(r, 'html.parser')

    article_text = ""                    
    tag = soup.find_all('div', class_="article-body")

    for t in tag:
        mytext = str(t)                      
        soup2 = BeautifulSoup(mytext, 'html.parser')  
        tag2 = soup2.find_all('p')

        for t2 in tag2:
            article_text += str(t2) 

    # Cleaning text
    article_text = re.sub("<[^>]*>", "", str(tag2), flags=re.MULTILINE)
    article_text = clean_article(article_text)
    article_text = article_text.replace('\xa0',' ')

    return article_text

urls = [
    'https://www.washingtonpost.com/world/national-security/on-russia-trump-and-his-top-national-security-aides-seem-to-be-at-odds/2017/04/18/13fdc832-23bf-11e7-bb9d-8cd6118e1409_story.html?hpid=hp_hp-top-table-main_trumprussia-740pm%3Ahomepage%2Fstory&utm_term=.57d1a4076ea0',
    'https://www.washingtonpost.com/world/national-security/on-north-korea-trump-administration-talks-tough-but-hopes-to-avoid-war/2017/04/18/96d15536-244a-11e7-bb9d-8cd6118e1409_story.html?hpid=hp_hp-top-table-main_usnorthkorea-920pm%3Ahomepage%2Fstory&utm_term=.5647e44a59dd',
    'https://www.washingtonpost.com/powerpost/georgia-house-race-stokes-gop-identity-crisis--and-opportunity-for-democrats/2017/04/18/a2231a48-242f-11e7-b503-9d616bd5a305_story.html?hpid=hp_hp-top-table-main_georgia-9am%3Ahomepage%2Fstory&utm_term=.fe02318f77b6'
    ]

n = 0
for url in urls:
    articles['twp_%02d' % n] = post_scraper(url)
    n += 1 

In [23]:
# For Associated Press - this works!

def apr_scraper(url):
    r = urllib.request.urlopen(url)
    soup = BeautifulSoup(r, 'html.parser')
    tags = soup.find_all("p", class_="ap-story-p")

    # Cleaning text
    article_text = re.sub("<[^>]*>", "", str(tags), flags=re.MULTILINE)
    article_text = article_text.replace('&amp;apos;s', '')
    article_text = clean_article(article_text)
    
    return article_text

urls = [
    'http://hosted.ap.org/dynamic/stories/E/EU_FRANCE_ELECTION?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-16-18-26',
    'http://hosted.ap.org/dynamic/stories/U/US_TRUMP_100_THE_PRESIDENCY?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-12-04-39',
    'http://hosted.ap.org/dynamic/stories/U/US_TRUMP_CONGRESS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-12-46-14',
    'http://hosted.ap.org/dynamic/stories/A/AS_NKOREA_DETAINED_AMERICAN?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-10-22-53',
    'http://hosted.ap.org/dynamic/stories/M/ML_ISRAEL_50_YEARS_LATER_PALESTINIAN_REFUGEES?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-02-33-58',
    'http://hosted.ap.org/dynamic/stories/U/US_OBIT_ERIN_MORAN_INOL-?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-08-09-59',
    'http://hosted.ap.org/dynamic/stories/S/STRUGGLING_VENEZUELANS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-12-16-21',
    'http://hosted.ap.org/dynamic/stories/A/AF_KENYA_NOVELIST_SHOT?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-13-45-09',
    'http://hosted.ap.org/dynamic/stories/U/US_FEMALE_GENITAL_MUTILATION_MIOL-?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-10-54-21',
    'http://hosted.ap.org/dynamic/stories/E/EU_FRANCE_ELECTION_WHY_IT_MATTERS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-04-23-12-45-03'
]


n = 0
for url in urls:
    articles['apr_%02d' % n] = apr_scraper(url)
    n += 1 

In [47]:
# Daily kos - this works!

def kos_scraper(url):
    r = urllib.request.urlopen(url)
    soup = BeautifulSoup(r, 'html.parser')
    tags = soup.find_all('p')

    # Cleaning text
    article_text = re.sub("<[^>]*>", "", str(tags), flags=re.MULTILINE)
    article_text = re.sub('\r\n*', '', article_text, flags=re.MULTILINE)
    article_text = article_text.replace('\xa0',' ')
    article_text = clean_article(article_text)
    
    return article_text

urls = [
    'http://www.dailykos.com/stories/2017/4/20/1654826/-Victory-Court-strikes-down-Texas-GOP-drawn-state-House-gerrymander-over-racial-discrimination',
    'http://www.dailykos.com/story/2017/04/23/1655584/-Macron-Le-Pen-Projected-Winners-of-Round-One-in-French-Election-Live-Stream-Coverage-in-English',
    'http://www.dailykos.com/story/2017/04/23/1655600/-How-our-Piratical-President-will-likely-mark-his-100th-day-in-office',
    'http://www.dailykos.com/story/2017/04/23/1655604/-US-Military-Keeps-Civilian-Casualties-Down-By-Calling-Every-Adult-Male-A-Combatant',
    'http://www.dailykos.com/story/2017/04/23/1655601/-What-Makes-White-Men-So-Powerful',
    'http://www.dailykos.com/story/2017/04/23/1655598/-Earth-Day',
    'http://www.dailykos.com/story/2017/04/23/1655540/-Government-shutdowns-A-relevant-little-history-lesson',
    'http://www.dailykos.com/story/2017/04/23/1655478/-So-Trump-wants-a-trade-war-But-is-pissed-when-Canada-makes-the-first-move',
    'http://www.dailykos.com/story/2017/04/23/1655597/-Chuck-Todd-asks-Dem-Leader-Nancy-Pelosi-a-dumb-question-she-slams-while-calling-out-Trump-s-weakness',
    'http://www.dailykos.com/story/2017/04/23/1655588/-Report-Ryan-giving-up-on-GOP-s-top-two-issues'
    ]

n = 0
for url in urls:
    articles['kos_%02d' % n] = kos_scraper(url)
    n += 1 

In [48]:
# Working output: dictionary of articles from NYT, Fox, Washington Post
for key in ['fox_00', 'nyt_00', 'twp_00', 'apr_00', 'kos_00']:
    print (key, ':',articles[key],'\n')

fox_00 :  president trump on tuesday signed an executive order that will make it harder for american tech companies to hire employees from foreign countries willing to work for less money than americans. trump signed the “buy american hire american” order during his visit to the snap on tools headquarters in kenosha wis. the president called the order “bold new steps” toward making good on his campaign promises to generate more jobs for out of work americans. trump called the order “a powerful signal to the world” that “finally puts america first.” “we are finally standing up for our workers and our companies” trump said. the order targets the h{DG} b visa program which allows u.s. companies to employ graduate level workers in specialty occupations like it engineering mathematics and science. among other changes the white house wants to end the h{DG} b lottery system and replace it with a merit based one   though it’s unclear exactly what criteria they would use.   “we’re going to swit

In [209]:
# non-working attempts

In [35]:
# Breitbart ; can't figure this out

url = 'http://www.breitbart.com/video/2017/04/20/trump-never-give-health-care-will-great-health-care/'
r = urllib.request.urlopen(url)
soup = BeautifulSoup(r, 'html.parser')
tags = soup.find_all("p", text = True)
article_text = re.sub("<[^>]*>", "", str(tags), flags=re.MULTILINE)
article_text = article_text.replace('<*>', '')
article_text

'[Trump said, “We’re doing very well on health care. We’ll see what happens. This is a great bill. There’s a great plan, and this will be great health care. It’s evolving. there was never a give-up. The press sort of reported there was like a give-up. There’s no give-up. We started. remember it took Obamacare 17 months.”, “I’ve really been negotiating this for two months, maybe even less than that because we had a 30-day period where we did lots of other things, the first 30 days, but this has really been two months. This is a continuation. The plan gets better and better and better, and it’s gotten really, really good and a lot of people are liking it a lot. We have a good chance of getting it soon. I’d like to say next week but I believe we will get it. Whether it’s next week or shortly thereafter.”, Send A Tip, Send A Tip, advertisement, Copyright © 2017 Breitbart, Copyright © 2017 Breitbart]'

In [142]:
# The New Yorker - this one doesn't work
url = 'http://www.newyorker.com/news/amy-davidson/fox-lost-bill-oreilly-but-it-still-has-donald-trump?intcid=mod-latest'
r = urllib.request.urlopen(url)
soup = BeautifulSoup(r, 'html.parser')
#tags = soup.find_all("p", text=True)
tags = soup.find_all ('div',class_= 'articleBody')
print (tags)

[<div class="articleBody" id="articleBody" itemprop="articleBody">
<figure class="main_column featured horizontal attachment-large img-expandable landscape" id="featured-item">
<a class="image" href="http://www.newyorker.com/wp-content/uploads/2017/04/Davidson-FoxLostBillOReillyButItStillHasDonaldTrump-1200.jpg" title="View larger"><img alt="Bill O’Reilly filled a need for Fox News, as it built its brand and its culture. But, in the Trump era, it doesn’t really require him for that anymore." class="horizontal attachment-large img-expandable landscape lazyload" data-id="3340600" data-ratio="1.500" itemprop="image" sizes="(max-width: 690px) 100vw, 690px" src="http://www.newyorker.com/wp-content/uploads/2017/04/Davidson-FoxLostBillOReillyButItStillHasDonaldTrump-690.jpg" srcset="http://www.newyorker.com/wp-content/uploads/2017/04/Davidson-FoxLostBillOReillyButItStillHasDonaldTrump-690.jpg 690w, http://www.newyorker.com/wp-content/uploads/2017/04/Davidson-FoxLostBillOReillyButItStillHasDon

In [46]:
# Daily kos - also not quite working
url = 'http://www.dailykos.com/stories/2017/4/20/1654826/-Victory-Court-strikes-down-Texas-GOP-drawn-state-House-gerrymander-over-racial-discrimination'
r = urllib.request.urlopen(url)
soup = BeautifulSoup(r, 'html.parser')
tags = soup.find_all('p')
article_text = re.sub("<[^>]*>", "", str(tags), flags=re.MULTILINE)
article_text = re.sub('\r\n*', '', article_text, flags=re.MULTILINE)
article_text = article_text.replace('\xa0',' ')
article_text = clean_article(article_text)
article_text


'late on thursday a federal district court struck down the state house map that texas republicans drew in {DG} on the grounds that it intentionally engaged in racial discrimination in violation of the voting rights act the {DG}th amendment and the “one person one vote” principle. this major voting rights victory could subsequently result in republican legislators having to draw a new map for the {DG} elections that would give black and latino voters the ability to elect their preferred candidates in more districts most likely democrats. thursday’s ruling follows two separate recent court decisions that invalidated the texas gop’s congressional map and strict voter id law and all three crucially held that republicans intentionally racially discriminated. this illicit racial intent could be grounds for forcing texas to seek justice department approval for all new voting law changes for up to ten years which it previously had to do until the supreme court struck down a key part of the vot

In [177]:
# CNN - not working
url = 'http://www.cnn.com/2017/04/19/opinions/trump-north-korea-everard-opinion/index.html'
r = urllib.request.urlopen(url)
soup = BeautifulSoup(r, 'html.parser')
soup.find_all("p")#,class_ = 'body__paragraph')#, id="storytext")

[<p class="edition-picker__radio-button-label">Set edition preference:</p>,
 <p class="edition-picker__radio-button-label">Set edition preference:</p>,
 <p class="edition-picker__radio-button-label">Set edition preference:</p>,
 <p class="metadata__byline"><span class="metadata__byline__author">By John Everard</span></p>,
 <p class="metadata__show"></p>,
 <p class="update-time">Updated 12:03 AM ET, Thu April 20, 2017 <span class="video__source top_source" id="js-pagetop_video_source"></span></p>,
 <p>Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.</p>,
 <p class="zn-body__paragraph"><q class="el-editorial-note">John Everard is a former British ambassador to North Korea. The opinions in this article belong solely to the author.</q></p>,
 <p class="zn-body__paragraph"><cite class="el-editorial-source"> (CNN)</cite>Saturday, the 105th birthday of Kim Il Sung, known as North Korea's Eternal President, was tense.  </p>,
 <p class="metadata__source">

In [91]:
articles.keys()

dict_keys(['twp_03', 'nyt_05', 'fox_01', 'fox_09', 'nyt_00', 'nyt_07', 'nyt_03', 'fox_05', 'twp_01', 'nyt_08', 'fox_06', 'nyt_04', 'fox_08', 'nyt_02', 'fox_07', 'fox_00', 'nyt_10', 'nyt_06', 'fox_02', 'fox_03', 'nyt_09', 'twp_02', 'fox_04', 'twp_00', 'nyt_01'])